# 导入模块

In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

# 连接数据库

In [2]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [4]:
# sql = ''' -- 订单逾期表
# select
# yoo.order_number, tmuic.id_card_address
# from db_rent.ya_order_overdue yoo
# -- 小程序用户身份证信息相关接口
# left join db_digua_business.t_member_user_id_card tmuic on tmuic.id_card_no=yoo.id_card_num
# '''
# df = query(sql)

In [5]:
# df[df.id_card_address.str.contains('惠东')==True]

# 获取数据

In [52]:
sql = '''
select
tprm.order_number, tmu.true_name, tmu.id_card_num, tol.delivery_detail_address, tod.new_actual_money, tprm.rent_received, ymos.sort, ymos.part_payment,ymos.refund_date
, ymos.reality_refund_date, om.merchant_name, tod.sku_attributes, pa.type, om.status
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_member_user tmu on tmu.id=om.user_id
left join db_digua_business.t_order_logistics tol on tol.order_id=om.id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
left join db_rent.ya_merchant_order_stages ymos on ymos.order_id=om.id
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
where ymos.sort<6
'''
df = query(sql)

# 清理数据

In [53]:
today = pd.to_datetime(datetime.now().date())
df = df[df.type!=4]
df = clean.drop_merchant(df)
df = clean.drop_rejected_merchant(df)
df.dropna(subset=['id_card_num'], inplace=True)
df = df[(df.refund_date<=today)&(df.status==4)]

In [54]:
today

Timestamp('2025-04-16 00:00:00')

In [60]:
id_4_list = ['3707', '3701', '3703', '3715']
id_4_dict = {
    '3707': '潍坊',
    '3701': '济南',
    '3703': '淄博',
    '3715': '聊城'
}
df.loc[:, 'id_card_4'] = df.id_card_num.apply(lambda x: x[:4])
df.loc[:, '首逾'] = np.where(df.sort==2, 
                        np.where(df.reality_refund_date.isna(), today-df.refund_date, 
                            np.where(df.reality_refund_date>df.refund_date, df.reality_refund_date-df.refund_date, 0)),0)
df.loc[:, '首逾'] = df.首逾.apply(lambda x: x.days)
df.loc[:, '逾期'] = np.where(df.reality_refund_date.isna(), today-df.refund_date, 0)
df.loc[:, '逾期'] = df.逾期.apply(lambda x: x.days)
df.loc[:, '剩余买断尾款'] = df.new_actual_money-df.rent_received
df.loc[:, '租期'] = df[df.part_payment>0].sort_values('sort', ascending=False).groupby('order_number').head(1).sort
df.loc[:, '租期'] = df.租期.ffill()
df.loc[:, '户籍地'] = df.id_card_4.apply(lambda x: id_4_dict[x] if x in id_4_list else x)

In [61]:
dff = df[df.part_payment>0].sort_values('sort', ascending=False).groupby('order_number').head(1)
# dff.loc[:, '租期'] = dff.sort

In [62]:
df[df.order_number=='A20240828113046117']

,order_number,true_name,id_card_num,delivery_detail_address,new_actual_money,rent_received,sort,part_payment,refund_date,reality_refund_date,merchant_name,sku_attributes,type,status,id_card_4,首逾,逾期,剩余买断尾款,租期,户籍地


# 首逾>60

In [63]:
df_sy = df[(df.id_card_4.isin(id_4_list))&(df.首逾>60)].drop_duplicates()
df_sy = df_sy[['order_number', 'true_name', 'id_card_num', 'delivery_detail_address', '户籍地', '租期', '剩余买断尾款']].rename(columns={'order_number': '订单号', 'true_name': '姓名', 'id_card_num': '身份证号', 'delivery_detail_address': '收货地址'})


,订单号,姓名,身份证号,收货地址,户籍地,租期,剩余买断尾款
26629,A20230925095506190,张国庆,370786199810263631,名仕雅居5号楼三单元502,潍坊,1.0,9647.91
28404,A20231005114350161,魏斌,370321199403233013,山东省-淄博市-桓台县-索镇街道恒生未来城小区27号楼一单元402室,淄博,1.0,12283.44
28564,A2023100608402969,左广彩,370782200308232020,东莞镇横山村,潍坊,1.0,12270.39
33194,A2023103013235547,赵迪,370302198608195117,山东省-淄博市-淄川区-双杨镇双凤生活区南区21号楼二单元502,淄博,1.0,10495.17
52318,A202403081103363,王新林,371521198910190756,聊城市东昌府区裕昌九州国际5号楼1单元1172,聊城,1.0,11913.88
77298,A2024062812072437,张兆鑫,371523200008236273,泺口办事处宿舍一号楼四单元701,聊城,1.0,11856.60
104888,A2024091115081353,曾鹏,37072419970210495X,潍坊绿地城3栋1单元1501,潍坊,1.0,12366.45
118748,A20241024174816254,汪兰超,371581199010146830,长阳镇京深路朱岗子村一区205室,聊城,1.0,12273.30
124633,A20241111153951319,李钦博,371502200604136058,东昌路159号正信招投标服务中心,聊城,1.0,10455.03
139388,A20241212094309161,朱志明,370786198904195110,梨园镇通州区新华联家园南区45号楼662,潍坊,1.0,10756.94


# 逾期>60

In [65]:
df_yq = df[(df.id_card_4.isin(id_4_list))&(df.逾期>60)].drop_duplicates(subset=['order_number'])
df_yq = df_yq[['order_number', 'true_name', 'id_card_num', 'delivery_detail_address', '户籍地', '租期', '剩余买断尾款']].rename(columns={'order_number': '订单号', 'true_name': '姓名', 'id_card_num': '身份证号', 'delivery_detail_address': '收货地址'})


(42, 7)

In [66]:
with pd.ExcelWriter('F:/需求/总需求/逾期户籍地.xlsx', engine='xlsxwriter') as writer:
    df_sy.to_excel(writer, sheet_name='首逾', index=False)
    df_yq.to_excel(writer, sheet_name='逾期', index=False)